# Sect 41: Recommendation Systems

- online-ds-ft-100719
- 03/24/20

# Questions


- 

## Note re: launching pyspark docker container

- To launch the pyspark docker container on a different port number besides 8888 (to avoid port conflicts):
    - Change the first port number to be something else (like 12345)
`
docker run -it -p 12345:8888 -v "filepath/to/folder":/home/jovyan/work --rm jupyter/pyspark-notebook`

    - Then launch url provided by server and change the URL in the address bar to use the port # you used (instead of 8888)

# Building a Movie Recommendation System in PySpark - Lab Code-along
![images of vhs tapes on shelf](img/movies.jpg)

## Introduction

In this last lab, we will implement a a movie recommendation system using Alternating Least Squares (ALS) in the Spark programming environment.<br> Spark's machine learning libraray `ml` comes packaged with a very efficient imeplementation of the ALS algorithm. 

The lab will require you to put into practice your spark programming skills for creating and manipulating pyspark DataFrames. We will go through a step-by-step process into developing a movie recommendation system using ALS and pyspark using the MovieLens Dataset.

Note: You are advised to refer to [PySpark Documentation](http://spark.apache.org/docs/2.2.0/api/python/index.html) heavily for completing this lab as it will introduce a few new methods. 

## Objectives

You will be able to:

* Identify the key components of the ALS 
* Demonstrate an understanding on how recommendation systems are being used for personalization of online services/products
* Parse and filter datasets into Spark DataFrame, performing basic feature selection
* ~~Run a brief hyper-parameter selection activity through a scalable grid search~~
* Train and evaluate the predictive performance of recommendation system
* Generate predictions from the trained model

## Building a Recommendation System

We have seen how recommender/Recommendation Systems have played an  integral parts in the success of Amazon (Books, Items), Pandora/Spotify (Music), Google (News, Search), YouTube (Videos) etc.  For Amazon these systems bring more than 30% of their total revenues. For Netflix service, 75% of movies that people watch are based on some sort of recommendation.

> The goal of Recommendation Systems is to find what is likely to be of interest to the user. This enables organizations to offer a high level of personalization and customer tailored services.

### We sort of get the concept

For online video content services like Netflix and Hulu, the need to build robust movie recommendation systems is extremely important. An example of recommendation system is such as this:

1.    User A watches Game of Thrones and Breaking Bad.
2.    User B performs a search query for Game of Thrones.
3.    The system suggests Breaking Bad to user B from data collected about user A.


This lab will guide you through a step-by-step process into developing such a movie recommendation system. We will use the MovieLens dataset to build a movie recommendation system using the collaborative filtering technique with Spark's Alternating Least Saqures implementation. After building that recommendation system, we will go through the process of adding a new user to the dataset with some new ratings and obtaining new recommendations for that user.

## Will Nightengale like Toy Story?

Collaborative filtering and matrix decomposition allows us to use the history of others ratings, along with the entire community of ratings, to answer that question.

![image1](img/collab.png)


## Person vs vegetable

It's important to realize that there are two sides to recommendation

![image2](img/item_user_based.png)

## Code for model

If we wanted, we could jump to the code right now to make this happen.

But would we understand it?
```python
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.recommendation import ALS, ALSModel

als = ALS(
    rank=10,
    maxIter=10,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating',
)

als_model = als.fit(movie_ratings)
```

## Documentation Station

Let's explore the [documentation](http://spark.apache.org/docs/2.4.3/api/python/pyspark.ml.html#module-pyspark.ml.recommendation) together to maybe get a better idea of what is happening. 

- which parameters make sense?
- which are completely foreign?

## Q: Whats the quickest way to pull up the full documentation for this new model/class?

- 

## A: `help`

In [1]:
## Whats a quick 
from pyspark.ml.recommendation import ALS, ALSModel
help(ALS)

Help on class ALS in module pyspark.ml.recommendation:

class ALS(pyspark.ml.wrapper.JavaEstimator, pyspark.ml.param.shared.HasCheckpointInterval, pyspark.ml.param.shared.HasMaxIter, pyspark.ml.param.shared.HasPredictionCol, pyspark.ml.param.shared.HasRegParam, pyspark.ml.param.shared.HasSeed, pyspark.ml.util.JavaMLWritable, pyspark.ml.util.JavaMLReadable)
 |  ALS(rank=10, maxIter=10, regParam=0.1, numUserBlocks=10, numItemBlocks=10, implicitPrefs=False, alpha=1.0, userCol='user', itemCol='item', seed=None, ratingCol='rating', nonnegative=False, checkpointInterval=10, intermediateStorageLevel='MEMORY_AND_DISK', finalStorageLevel='MEMORY_AND_DISK', coldStartStrategy='nan')
 |  
 |  Alternating Least Squares (ALS) matrix factorization.
 |  
 |  ALS attempts to estimate the ratings matrix `R` as the product of
 |  two lower-rank matrices, `X` and `Y`, i.e. `X * Yt = R`. Typically
 |  these approximations are called 'factor' matrices. The general
 |  approach is iterative. During each iter

## Rank

What's all this rank of the factorization business?<br>
[the source code documentation](https://github.com/apache/spark/blob/master/mllib/src/main/scala/org/apache/spark/mllib/recommendation/ALS.scala) describes that variable as the "Rank of the feature matrices"

## Assumptions

Matrix decomposition is built on the theory that every individual (user, movie) score is actually the **dot product** of two separate vectors:
- user characteristics 
- movie characteristics

Wait, do you mean like gender, whether the movie is sci-fi or action? Do we have that data?

![beyonce-gif](img/beyonce.gif)

## Huh?
![what](img/what.gif)

## The hidden matricies 
![image4](img/matrix_decomp.png)

## Embeddings

Embeddings are low dimensional hidden factors for items and users.

For e.g. say we have 5 dimensional (i.e., **rank** = 5) embeddings for both items and users (5 chosen randomly, this could be any number - as we saw with PCA and dim. reduction).

For user-X & movie-A, we can say those 5 numbers might represent 5 different characteristics about the movie e.g.:

- How much movie-A is political
- How recent is the movie
- How much special effects are in movie A
- How dialogue driven is the movie
- How linear is the narrative in the movie

In a similar way, 5 numbers in the user embedding matrix might represent:

- How much does user-X like sci-fi movies
- How much does user-X like recent movies … and so on.

But we have *no actual idea* what those factors actually represent.

### If we knew the feature embeddings in advance, it would look something like this:

In [3]:
import numpy as np

# the original matrix of rankings
R = np.array([[2, np.nan, np.nan, 1, 4],
       [5, 1, 2, np.nan, 2],
       [3, np.nan, np.nan, 3, np.nan],
       [1, np.nan, 4, 2, 1]])

# users X factors
P =np.array([[-0.63274434,  1.33686735, -1.55128517],
       [-2.23813661,  0.5123861 ,  0.14087293],
       [-1.0289794 ,  1.62052691,  0.21027516],
       [-0.06422255,  1.62892864,  0.33350709]])

# factors X items
Q = np.array([[-2.09507374,  0.52351075,  0.01826269],
       [-0.45078775, -0.07334991,  0.18731052],
       [-0.34161766,  2.46215058, -0.18942263],
       [-1.0925736 ,  1.04664756,  0.69963111],
       [-0.78152923,  0.89189076, -1.47144019]])

What about that `np.nan` in the third row, last column? How will that item be reviewed by that user?

In [5]:
print(P[2])
print()
print(Q.T[:,4])
P[2].dot(Q.T[:,4])

[-1.0289794   1.62052691  0.21027516]

[-0.78152923  0.89189076 -1.47144019]


1.9401031341455333

## Wait, I saw a transpose in there - what's the actual formula?

Terms:<br>
$R$ is the full user-item rating matrix

$P$ is a matrix that contains the users and the k factors represented as (user,factor)

$Q^T$ is a matrix that contains the items and the k factors represented as

$r̂_{u,i}$ represents our prediction for the true rating $r_{ui}$ In order to get an individual rating, you must take the dot product of a row of P and a column of Q

for the entire matrix:
$$ R = PQ^T $$ 

or for individual ratings

$$r̂_{u,i}=q_i^⊤p_u $$ 





### Let's get the whole matrix!

In [6]:
P.dot(Q.T)

array([[ 1.99717984, -0.10339773,  3.80157388,  1.00522135,  3.96947118],
       [ 4.95987359,  0.99772807,  1.9994742 ,  3.08017572,  1.99887552],
       [ 3.00799117,  0.38437256,  4.30166793,  2.96747131,  1.94010313],
       [ 0.99340337, -0.02806164,  3.96943336,  2.00841398,  1.01228247]])

### Look at those results

Are they _exactly_ correct?
![check](img/check.gif)

## ALS benefit: Loss Function

The Loss function $L$ can be calculated as:

$$ \large L = \sum_{u,i ∈ \kappa}(r_{u,i}− q_i^T p_u)^2 + λ( ||q_i||^2 + |p_u||^2)$$

Where $\kappa$ is the set of (u,i) pairs for which $r_{u,i}$ is known.

To avoid overfitting, the loss function also includes a regularization parameter $\lambda$. We will choose a $\lambda$ to minimize the square of the difference between all ratings in our dataset $R$ and our predictions.

There's the **least squares** part of ALS, got it!

## So now we use gradient decent, right?

![incorrect](img/incorrect.gif)

### Here comes the alternating part

ALS alternates between holding the $q_i$'s constant and the $p_u$'s constant. 

While all $q_i$'s are held constant, each $p_u$ is computed by solving the least squared problem.<br>
After that process has taken place, all the $p_u$'s are held constant while the $q_i$'s are altered to solve the least squares problem, again, each independently.<br> 
This process repeats many times until you've reached convergence (ideally).

### Changing Loss function:

First let's assume first the item vectors are fixed, we first solve for the user vectors:

$$p_u=(\sum{r{u,i}\in r_{u*}}{q_iq_i^T + \lambda I_k})^{-1}\sum_{r_{u,i}\in r_{u*}}{r_{ui}{q_{i}}}$$__
Then we hold the user vectors constant and solve for the item vectors

$$q_i=(\sum{r{u,i}\in r_{i*}}{p_up_u^T + \lambda I_k})^{-1}\sum_{r_{u,i}\in r_{u*}}{r_{ui}{p_{u}}}$$__
This process repeats until convergence

# Review
What levers do we have available to adjust?
![lever-choice](img/levers.jpeg)

- Pros and cons of large rank?
- Pros and cons of lambda size?
- Iterations?

# Enough - let's get to the data

### Importing the Data
To begin with:
* initialize a SparkSession object
* import the dataset found at './data/ratings.csv' into a pyspark DataFrame

In [7]:
# import necessary libraries
!env | grep PYTHONPATH

PYTHONPATH=/usr/local/spark/python:/usr/local/spark/python/lib/py4j-0.10.7-src.zip


In [8]:
import pyspark

spark = (pyspark.sql.SparkSession.builder 
  .master("local[*]")
  .getOrCreate())

In [9]:
!ls data/

movies.csv  ratings.csv


In [10]:
!file data/ratings.csv

/bin/sh: 1: file: not found


In [11]:
!head data/ratings.csv

In [12]:
# read in the dataset into pyspark DataFrame
movie_ratings = spark.read.csv('data/ratings.csv',
                               inferSchema=True,
                               header=True)

Check the data types of each of the values to ensure that they are a type that makes sense given the column.

In [13]:
movie_ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



But if they were ever incorrectly assigned, here's how we fix it:

In [14]:
from pyspark.sql.types import (
    ArrayType,
    AtomicType,
    BinaryType,
    BooleanType,
    ByteType,
    CloudPickleSerializer,
    DataType,
    DataTypeSingleton,
    DateConverter,
    DateType,
    DatetimeConverter,
    DecimalType,
    DoubleType,
    FloatType,
    FractionalType,
    IntegerType,
    IntegralType,
    JavaClass,
    LongType,
    MapType,
    NullType,
    NumericType,
    Row,
    ShortType,
    SparkContext,
    StringType,
    StructField,
    StructType,
    TimestampType,
    UserDefinedType,
)

In [15]:
schema = StructType(
    [
        StructField('userId', IntegerType()),
        StructField('movieId', IntegerType()),
        StructField('rating', FloatType()),
        StructField('timestamp', LongType()),
    ]
)

In [16]:
# read in the dataset into pyspark DataFrame
movie_ratings = spark.read.csv('data/ratings.csv',
                               inferSchema=False,
                               schema=schema,
                               header=True)

In [14]:
movie_ratings.persist()

DataFrame[userId: int, movieId: int, rating: float, timestamp: bigint]

In [17]:
movie_ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: long (nullable = true)



In [18]:
type(movie_ratings)

pyspark.sql.dataframe.DataFrame

In [19]:
movie_ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



We aren't going to need the time stamp, so we can go ahead and remove that column.

In [22]:
movie_ratings.select(movie_ratings.columns[:-1])

DataFrame[userId: int, movieId: int, rating: float]

In [24]:
movie_ratings = movie_ratings.drop('timestamp')#what do we put here?
movie_ratings.columns

['userId', 'movieId', 'rating']

### Fitting the Alternating Least Squares Model

Because this dataset is already preprocessed for us, we can go ahead and fit the Alternating Least Squares model.

* Use the randomSplit method on the pyspark DataFrame to separate the dataset into a training and test set
* Import the ALS module from pyspark.ml.recommendation.
* Fit the Alternating Least Squares Model to the training dataset. Make sure to set the userCol, itemCol, and ratingCol to the appropriate names given this dataset. Then fit the data to the training set and assign it to a variable model. 

### Train-Test Split

Split the data into training and testing sets.

In [25]:
from pyspark.ml.recommendation import ALS, ALSModel

als = ALS(
    rank=10,
    maxIter=10,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating',
)

In [27]:
## Train test split

train_data, test_data = movie_ratings.randomSplit([0.8,0.2])

print("Training Size: {}, Testing Size: {}".format(train_data.count(), test_data.count()))
length = movie_ratings.count()
print("Training Size: {}, Testing Size: {}".format(train_data.count()/length, test_data.count()/length))

Training Size: 80590, Testing Size: 20246
Training Size: 0.7992185330635884, Testing Size: 0.2007814669364116


In [28]:
# Build the recommendation model using ALS on the training data
# fit the ALS model to the training set

als_model = als.fit(train_data)

Now you've fit the model, and it's time to evaluate it to determine just how well it performed.

* Import `RegressionEvalutor` from pyspark.ml.evaluation [documentation](http://spark.apache.org/docs/2.4.3/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator)
* Generate predictions with your model for the train and test set by using the `transform` method on your ALS model
* Evaluate your model and print out the RMSE from your test set [options for evaluating regressors](http://spark.apache.org/docs/2.4.3/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator.metricName)

In [30]:
from pyspark.ml.evaluation import RegressionEvaluator

train_predictions = als_model.transform(train_data)
test_predictions = als_model.transform(test_data)


In [31]:
train_predictions.persist()
test_predictions.persist()

DataFrame[userId: int, movieId: int, rating: float, prediction: float]

In [32]:
movie_ratings.show(1)


+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
+------+-------+------+
only showing top 1 row



In [33]:
train_predictions.show(1)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   191|    148|   5.0| 4.9222593|
+------+-------+------+----------+
only showing top 1 row



In [34]:
test_predictions.show(1)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   602|    471|   4.0| 3.6176019|
+------+-------+------+----------+
only showing top 1 row



In [36]:
evaluator_rmse = RegressionEvaluator(predictionCol='prediction',labelCol='rating',metricName='rmse')
evaluator_r2 = RegressionEvaluator(predictionCol='prediction',labelCol='rating',metricName='r2')

In [37]:
evaluator_rmse.evaluate(train_predictions)

0.563142842852337

In [38]:
evaluator_r2.evaluate(train_predictions)

0.707895909277146

In [39]:
evaluator_rmse.evaluate(test_predictions)

nan

In [43]:
evaluator_r2.evaluate(test_predictions)

nan

> **Warning!**
Spark Considers Null Values and NaN (Not a Number) different entities. See the code below, where the NaN values from our prediction are not filtered out. See this [stack overflow thread](https://stackoverflow.com/questions/43882699/differences-between-null-and-nan-in-spark-how-to-deal-with-it#43883025) for further details.

In [45]:
test_predictions.filter(test_predictions.prediction.isNotNull()).show(2500)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   602|    471|   4.0| 3.6176019|
|    91|    471|   1.0| 2.8846242|
|   409|    471|   3.0| 3.6853895|
|   599|    471|   2.5| 2.8531346|
|   218|    471|   4.0| 2.3907223|
|   273|    471|   5.0| 4.4479513|
|   312|    471|   4.0| 3.8404975|
|   608|    833|   0.5| 2.0129232|
|   159|   1088|   4.0| 2.9258795|
|   599|   1088|   2.5| 2.5270731|
|    47|   1088|   4.0|  2.578249|
|   479|   1088|   4.0| 3.2128026|
|   489|   1088|   4.5| 3.2042365|
|   387|   1088|   1.5| 2.6286628|
|   583|   1088|   3.5| 3.5661032|
|   594|   1088|   4.5| 4.5391374|
|   226|   1088|   1.0|  3.391787|
|   221|   1088|   3.0| 3.2302706|
|   600|   1088|   3.5| 2.8282094|
|   104|   1088|   3.0|   3.61055|
|   462|   1238|   3.5|  4.174812|
|   312|   1342|   4.0| 2.6373682|
|   600|   1342|   2.5|  2.600164|
|    34|   1580|   2.5| 2.9460835|
|   274|   1580|   3.0| 3.1139693|
|   474|   1580|   4

In [49]:
from pyspark.sql.functions import isnan

evaluator_rmse.evaluate(test_predictions.where(isnan('prediction')))

nan

In [50]:
print(test_predictions.count())
test_predictions = test_predictions.na.drop()
print(test_predictions.count())
evaluator_rmse.evaluate(test_predictions), evaluator_r2.evaluate(test_predictions)

20246
19393


(0.8841781267461247, 0.2777021601151217)

### Important Question

Will Billy like movie m?

In [54]:
recs = als_model.recommendForAllUsers(numItems=10)
recs[recs['userId']==10].first()['recommendations']

[Row(movieId=3086, rating=4.750299453735352),
 Row(movieId=2693, rating=4.665172576904297),
 Row(movieId=8869, rating=4.646908760070801),
 Row(movieId=71579, rating=4.640575408935547),
 Row(movieId=140110, rating=4.455325126647949),
 Row(movieId=113275, rating=4.434729099273682),
 Row(movieId=1939, rating=4.420477390289307),
 Row(movieId=1962, rating=4.418495178222656),
 Row(movieId=51705, rating=4.414510250091553),
 Row(movieId=25906, rating=4.387438774108887)]

In [58]:
user_factors = als_model.userFactors
item_factors = als_model.itemFactors

In [59]:
billy_row = user_factors[user_factors['id'] == 10].first()
billy_factors = np.array(billy_row['features'])

In [60]:
m_row = item_factors[item_factors['id'] == 296].first()
m_factors = np.array(m_row['features'])

In [61]:
billy_factors

array([-0.1806546 , -0.99121284,  0.62874794, -0.78728271,  0.4058933 ,
        0.2127157 , -0.78479856, -1.33854759, -0.56726921, -0.41247904])

In [62]:
m_factors

array([ 0.56507516, -1.30459714,  1.0836612 , -1.32220554,  0.76590437,
        0.6669606 ,  0.33789819,  0.6742844 , -0.1576782 ,  0.48142102])

In [63]:
billy_factors @ m_factors

2.0892238897030158

In [65]:
predictions = als_model.transform(movie_ratings)

In [66]:
billy_preds = predictions[predictions['userId'] == 10]

In [67]:
billy_preds.sort('movieId').show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|    10|    296|   1.0| 2.0892239|
|    10|    356|   3.5| 3.6648097|
|    10|    588|   4.0| 3.0030289|
|    10|    597|   3.5|  3.001352|
|    10|    912|   4.0| 3.2146215|
|    10|   1028|   0.5| 2.4770858|
|    10|   1088|   3.0| 3.0432448|
|    10|   1247|   3.0| 2.3167267|
|    10|   1307|   3.0| 2.6196175|
|    10|   1784|   3.5| 3.3164616|
|    10|   1907|   4.0| 3.2249336|
|    10|   2571|   0.5| 3.2154198|
|    10|   2671|   3.5| 3.5114944|
|    10|   2762|   0.5|  2.546587|
|    10|   2858|   1.0| 2.1348896|
|    10|   2959|   0.5| 2.3971066|
|    10|   3578|   4.0|  3.501969|
|    10|   3882|   3.0| 3.1212914|
|    10|   4246|   3.5| 2.9358337|
|    10|   4306|   4.5| 4.0330334|
+------+-------+------+----------+
only showing top 20 rows



In [68]:
!grep 296 < data/movies.csv

## Okay, what *will* Billy like?

In [69]:
recs = als_model.recommendForAllUsers(numItems=10)

In [70]:
recs[recs['userId']==10].first()['recommendations']

[Row(movieId=3086, rating=4.750299453735352),
 Row(movieId=2693, rating=4.665172576904297),
 Row(movieId=8869, rating=4.646908760070801),
 Row(movieId=71579, rating=4.640575408935547),
 Row(movieId=140110, rating=4.455325126647949),
 Row(movieId=113275, rating=4.434729099273682),
 Row(movieId=1939, rating=4.420477390289307),
 Row(movieId=1962, rating=4.418495178222656),
 Row(movieId=51705, rating=4.414510250091553),
 Row(movieId=25906, rating=4.387438774108887)]

In [71]:
!grep 3086 < data/movies.csv

## Objective Review

* Identify the key components of the ALS 
* Demonstrate an understanding on how recommendation systems are being used for personalization of online services/products
* Parse and filter datasets into Spark DataFrame, performing basic feature selection
* ~~Run a brief hyper-parameter selection activity through a scalable grid search~~
* Train and evaluate the predictive performance of recommendation system
* Generate predictions from the trained model

## Some great technical resources:

- [Stanford ALS Theory and Implementation](http://stanford.edu/~rezab/classes/cme323/S15/notes/lec14.pdf)
- [The Netflix Recommendation Project](https://www.netflixprize.com/assets/GrandPrize2009_BPC_BellKor.pdf)